In [2]:
print("thx max")
import matplotlib.pyplot as plt
import os
import hist
import mplhep as hep
import torch
import numpy as np
import hist
from hist import Hist
import traceback
from helpers import mass
import pandas as pd
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable

from helpers import *
import os
from scipy import stats
import datetime
import pandas as pd
import traceback
import time
from torch import nn
gen_step = 0

# train mode
import nflows as nf
from nflows.utils.torchutils import create_random_binary_mask
from nflows.transforms.base import CompositeTransform
from nflows.transforms.coupling import *
from nflows.nn import nets
from nflows.flows.base import Flow
from nflows.flows import base
from nflows.transforms.coupling import *
from nflows.transforms.autoregressive import *

from pytorch_lightning.callbacks import ModelCheckpoint
# from comet_ml import Experiment

import pytorch_lightning as pl
import os
# from plotting import plotting
from torch.nn import functional as FF
import traceback
import os

import pytorch_lightning as pl
from torch.optim.lr_scheduler import OneCycleLR,ReduceLROnPlateau,ExponentialLR
import torch
from torch import nn
from torch.nn import functional as FF
import numpy as np
from jetnet.evaluation import w1p, w1efp, w1m, cov_mmd,fpnd
import mplhep as hep
import hist
from hist import Hist
from pytorch_lightning.loggers import TensorBoardLogger
from collections import OrderedDict

from helpers import *

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import time
print("good boy")
# from torch.nn import MultiheadAttention,TransformerEncoder,TransformerEncoderLayer
def mass(data,canonical=False):
    if canonical:
        n_dim=data.shape[1]
        p=data.reshape(-1,n_dim//3,3)
        px=p[:,:,0]
        py=p[:,:,1]
        pz=p[:,:,2]
        
       
    else:
        n_dim=data.shape[1]
        p=data.reshape(-1,n_dim//3,3)
        px=torch.cos(p[:,:,1])*p[:,:,2]
        py=torch.sin(p[:,:,1])*p[:,:,2]
        pz=torch.sinh(p[:,:,0])*p[:,:,2]
    px=torch.clamp(px,min=-100,max=100)
    py=torch.clamp(py,min=-100,max=100)
    pz=torch.clamp(pz,min=-100,max=100)
    E=torch.sqrt(px**2+py**2+pz**2)
    E=E.sum(axis=1)**2
    p=px.sum(axis=1)**2+py.sum(axis=1)**2+pz.sum(axis=1)**2
    m2=E-p
    # if m2.isnan().any():
    #     print("px:{} py:{} pz:{} ".format(px.abs().max(),py.abs().max(),pz.abs().max()))
    # assert m2.isnan().sum()==0  
    return torch.sqrt(torch.max(m2,torch.zeros(len(E)).to(E.device)))
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)
class plotting():
    '''This is a class that takes care of  plotting steps in the script,
        It is initialized with the following arguments:
        true=the simulated data, note that it needs to be scaled
        gen= Generated data , needs to be scaled
        step=The current step of the training, this is need for tensorboard
        model=the model that is trained, a bit of an overkill as it is only used to access the losses
        config=the config used for training
        logger=The logger used for tensorboard logging'''
    def __init__(self,true,gen,gen_corr,config,step,model=None,logger=None,):
        self.config=model.config
        self.n_dim=self.config["n_dim"]
        self.gen=gen.numpy()
        self.gen_corr=gen_corr.numpy()
        self.test_set=true.numpy()
        self.step=step
        self.model=model
        if logger is not None:
            self.summary=logger
   

    def plot_2d(self,save=False):
        #This creates a 2D histogram of the inclusive distribution for all 3 feature combinations
        #Inclusive means that is the distribution of pt of all particles per jet and sample
        #if save, the histograms are logged to tensorboard otherwise they are shown
        data=self.test_set[:,:self.n_dim].reshape(-1,3)
        gen=self.gen[:,:self.n_dim].reshape(-1,3)
        labels=[r"$\eta^{rel}$",r"$\phi^{rel}$",r"$p_T^{rel}$"]
        names=["eta","p3hi","pt"]
        for index in [[0,1],[0,2],[1,2]]:

            fig,ax=plt.subplots(ncols=2,figsize=(16, 8))
            _,x,y,_=ax[0].hist2d(data[:,index[0]],data[:,index[1]],bins=30)
            
            if index[1]==2:
                
                y = np.logspace(np.log(y[0]),np.log(y[-1]),len(y))
                ax[0].hist2d(data[:,index[0]],gen[:,index[1]],bins=[x,y])
            ax[1].hist2d(gen[:,index[0]],gen[:,index[1]],bins=[x,y])
            plt.tight_layout(pad=2)
            ax[0].set_xlabel( labels[index[0]])
            ax[0].set_ylabel( labels[index[1]])
            
            ax[0].set_title("Data")
            ax[1].set_xlabel( labels[index[0]])
            ax[1].set_ylabel( labels[index[1]])
            
            ax[1].set_title("Gen")
           
            if save:
                self.summary.add_figure("2d{}-{}".format(names[index[0]],names[index[1]]),fig,global_step=self.step)
                
                # self.summary.close()
            else:
                plt.show()
 
        
    def plot_mass(self,m,m_t,m_c,save=False,quantile=False,bins=15,plot_vline=True):
        #This creates a histogram of the inclusive distributions and calculates the mass of each jet
        #and creates a histogram of that
        #if save, the histograms are logged to tensorboard otherwise they are shown
        #if quantile, this also creates a histogram of a subsample of the generated data, 
        # where the mass used to condition the flow is in the first 10% percentile of the simulated mass dist
        i=0


        for v,name in zip(["eta","phi","pt","m"],[r"$\eta^{rel}$",r"$\phi^{rel}$",r"$p_T^{rel}$",r"$m_T^{rel}$"]):
            
            if v!="m":
                a=np.quantile(self.test_set[:,i],0.001)
                b=np.quantile(self.test_set[:,i],0.999)
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                h3=hist.Hist(hist.axis.Regular(bins,a,b))
                
                h.fill(self.test_set[:,i])
                h2.fill(self.gen[:,i])
                h3.fill(self.gen_corr[:,i])
                i+=1
            else:
                a=np.quantile(m_t,0.001)
                b=np.quantile(m_t,0.999)
                h=hist.Hist(hist.axis.Regular(bins,a,b))
                h2=hist.Hist(hist.axis.Regular(bins,a,b))
                h3=hist.Hist(hist.axis.Regular(bins,a,b))
                bins = h.axes[0].edges
                h.fill(m_t)
                h2.fill(m)
                h3.fill(m_c)
            fig,ax=plt.subplots(2,1,gridspec_kw={'height_ratios': [3, 1]},figsize=(8,8))
            
#             hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            try:
                main_ax_artists, sublot_ax_arists = h.plot_ratio(
                    h3,
                    ax_dict={"main_ax":ax[0],"ratio_ax":ax[1]},
                    rp_ylabel=r"Ratio",
                    rp_num_label="MC Simulated",
                    rp_denom_label="Flow Generated+Corrected",
                    rp_uncert_draw_type="line",  # line or bar
                )
                
                h2.plot(ax=ax[0],label="Flow Generated")
                ax[0].set_xlabel("")
#                 if quantile and v=="m" and plot_vline:
#                     ax[0].hist(m[m_t<np.quantile(m_t,0.1)],histtype='step',bins=bins,alpha=1,color="red",label="10% quantile Flow Generated",hatch="/")
#                     ax[0].vlines(np.quantile(m_t,0.1),0,np.max(h[:]),color="red",label='10% quantile MC Simulated')
                    
                ax[1].set_ylim(0.25,2)
                ax[0].set_xlim(a,b)
                ax[0].legend()
                ax[1].set_xlim(a,b)
#                 if v!="m":
#                     ax[0].legend(["Flow Generated","MC Simulated"])
#                 elif plot_vline:
#                     ax[0].legend(["Flow Generated","MC Simulated"] )
            except:
                print("mass plot failed reverting to simple plot mass bins")
                traceback.print_exc()
                plt.close()
                plt.figure()
                _,b,_=plt.hist(m_t,15,label="Sim",alpha=0.5)
                plt.hist(m,b,label="Gen",alpha=0.5)
                plt.legend()  
           # hep.cms.label(data=False,lumi=None ,year=None,rlabel="",llabel="Private Work",ax=ax[0] )
            ax[1].set_xlabel(name,fontsize=28)
            ax[0].set_ylabel("Counts",fontsize=28)
            ax[1].set_ylabel("Ratio",fontsize=28)
            
            plt.tight_layout(pad=2)
            if save:
                if v!="m":
                     self.summary.add_figure("inclusive"+v,fig,self.step)
                else:
                    self.summary.add_figure("jet_mass",fig,self.step)
    #             print("added figure")
    #             self.summary.close()
            else:
                plt.show()

    
    
    def plot_correlations(self,save=True):
        #Plots correlations between all particles for i=0 eta,i=1 phi,i=2 pt
        self.plot_corr(i=0,save=save)
        self.plot_corr(i=1,save=save)
        self.plot_corr(i=2,save=save)

    def plot_corr(self,i=0,names=["$\eta^{rel}$","$\phi^{rel}$","$p_T$"],save=True):
        if i==2:
            c=1
        else:
            c=.25
        df_g=pd.DataFrame(self.gen[:,:self.n_dim][:,range(i,90,3)])
        df_h=pd.DataFrame(self.test_set[:,:self.n_dim][:,range(i,90,3)])
        
        fig,ax=plt.subplots(ncols=2,figsize=(15,7.5))
        corr_g = ax[0].matshow(df_g.corr())
        corr_g.set_clim(-c,c)
        divider = make_axes_locatable(ax[0])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar=fig.colorbar(corr_g,cax=cax)

        corr_h = ax[1].matshow(df_h.corr())
        corr_h.set_clim(-c,c)
        divider = make_axes_locatable(ax[1])

        cax2 = divider.append_axes('right', size='5%', pad=0.05)
        cbar=fig.colorbar(corr_h,cax=cax2)
        plt.suptitle("{} Correlation between Particles".format(names[i]),fontsize=38)
        ax[0].set_title("Flow Generated",fontsize=34)
        ax[1].set_title("MC Simulated",fontsize=28)
        ax[0].set_xlabel("Particles",fontsize=28)
        ax[0].set_ylabel("Particles",fontsize=28)
        ax[1].set_xlabel("Particles",fontsize=28)
        ax[1].set_ylabel("Particles",fontsize=28)
        ax[0].set_xticks([])
        ax[1].set_xticks([])
        ax[0].set_yticks([])
        ax[1].set_yticks([])
        if save:
                title=["corr_eta","corr_phi","corr_pt"]
                self.summary.add_figure(title[i],fig,self.step)
                
    #             self.summary.close()
        else:
                plt.show()
from torch import optim
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

            def __init__(self, optimizer, warmup, max_iters):
                self.warmup = warmup
                self.max_num_iters = max_iters
                super().__init__(optimizer)

            def get_lr(self):
                lr_factor = self.get_lr_factor(epoch=self.last_epoch)
                return [base_lr * lr_factor for base_lr in self.base_lrs]

            def get_lr_factor(self, epoch):
                lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
                if epoch <= self.warmup:
                    lr_factor *= epoch * 1.0 / self.warmup
                return lr_factor
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)
    def inverse_transform(self,values):
        return (values *self.std)+self.mean
    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)
    def to(self,dev):
        self.std=self.std.to(dev)
        self.mean=self.mean.to(dev)
        return self
  
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
class JetNetDataloader(pl.LightningDataModule):
    '''This is more or less standard boilerplate coded that builds the data loader of the training
       one thing to note is the custom standard scaler that works on tensors
       Currently only jets with 30 particles are used but this maybe changes soon'''
    def __init__(self,config): 
        super().__init__()
        self.config=config
        self.n_dim=config["n_dim"]
        self.n_part=config["n_part"]
        self.batch_size=config["batch_size"]
    def setup(self,stage):
    # This just sets up the dataloader, nothing particularly important. it reads in a csv, calculates mass and reads out the number particles per jet
    # And adds it to the dataset as variable. The only important thing is that we add noise to zero padded jets
        data_dir=os.environ["HOME"]+"/JetNet_NF/train_{}_jets.csv".format(self.config["parton"])
        data=pd.read_csv(data_dir,sep=" ",header=None)
        jets=[]
        limit=int(self.config["limit"]*1.1)
        for njets in range(1,31):
            masks=np.sum(data.values[:,np.arange(3,120,4)],axis=1)
            df=data.loc[masks==njets,:]
            df=df.drop(np.arange(3,120,4),axis=1)
            df["n"]=njets
            if len(df)>100:
                jets.append(df[:self.config["limit"]])
        #stacking together differnet samples with different number particles per jet
        self.n=torch.empty((0,1))
        self.data=torch.empty((0,self.n_dim*self.n_part))
        for i in range(len(jets)):
            x=torch.tensor(jets[i].values[:,:self.n_dim*self.n_part]).float()
            n=torch.tensor(jets[i]["n"].values).float()
            self.data=torch.vstack((self.data,x))
            self.n=torch.vstack((self.n.reshape(-1,1),n.reshape(-1,1)))        
        
      
        # calculating mass per jet
#         self.m=mass(self.data[:,:self.n_dim]).reshape(-1,1)  
      # Adding noise to zero padded jets.
        for i in torch.unique(self.n):
            i=int(i)
            self.data[self.data[:,-1]==i,3*i:90]=torch.normal(mean=torch.zeros_like(self.data[self.data[:,-1]==i,3*i:90]),std=1).abs()*1e-7
        #standard scaling 
        self.scaler=StandardScaler()
#         self.data=torch.hstack((self.data,self.m))        
        self.scaler.fit(self.data)
        self.data=self.scaler.transform(self.data)
#         self.min_m=self.scaler.transform(torch.zeros((1,self.n_dim+1)))[0,-1]
# #         self.data=torch.hstack((self.data,self.n))
        
#         #calculating mass dist in different bins, this is needed for the testcase where we need to generate the conditoon
#         if self.config["variable"]:
#             self.mdists={}
#             for i in torch.unique(self.n):
#                 self.mdists[int(i)]=F(self.data[self.n[:,0]==i,-2])    
        self.data,self.test_set=train_test_split(self.data.cpu().numpy(),test_size=0.3)
        
#         self.n_train=self.data[:,-1]
#         self.n_test=self.test_set[:,-1]
        
            
        self.test_set=torch.tensor(self.test_set).float()
        self.data=torch.tensor(self.data).float()
#         assert self.data.shape[1]==92
        assert (torch.isnan(self.data)).sum()==0
    def train_dataloader(self):
        return DataLoader(self.data, batch_size=self.batch_size,drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=len(self.test_set),drop_last=True)

thx max
good boy


 # STOP
 

In [5]:
from torch.nn.functional import leaky_relu,sigmoid

class Disc(nn.Module):
    def __init__(self,n_dim=3,l_dim=10,hidden=300,num_layers=3,num_heads=1,n_part=2,fc=True):
        super().__init__()
        self.hidden_nodes=hidden
        self.n_dim=n_dim
        self.l_dim=l_dim
        self.n_part=n_part
        self.fc=fc

        
        if fc:
            self.l_dim*=n_part 
            self.embbed_flat=nn.Linear(n_dim*n_part,l_dim)
            self.flat_hidden=nn.Linear(l_dim,hidden)
            self.flat_hidden2=nn.Linear(hidden,hidden)
            self.flat_hidden3=nn.Linear(hidden,hidden)
            self.flat_out=nn.Linear(hidden,1)
        else:
            self.embbed=nn.Linear(n_dim,l_dim)
            self.encoder=nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=l_dim,nhead=num_heads,dim_feedforward=hidden,dropout=0.1,norm_first=True,
                                       activation="relu",batch_first=True) ,num_layers=num_layers)
            self.hidden=nn.Linear(l_dim,hidden)
            self.hidden2=nn.Linear(hidden,hidden)
            self.out=nn.Linear(hidden,1)

    def forward(self,x):

        if self.fc==True:
            x=x.reshape(len(x),self.n_dim*self.n_part)
            x=self.embbed_flat(x)
            x=leaky_relu(self.flat_hidden(x),0.2)
            x=leaky_relu(self.flat_hidden2(x),0.2)
            x=self.flat_out(x)
        else:
            x=self.embbed(x)
#             x=torch.concat((torch.ones_like(x[:,0,:]).reshape(len(x),1,-1),x),axis=1)
            x=self.encoder(x)
            x=torch.sum(x,axis=1)
#             x=x[:,0,:]
            
            x=leaky_relu(self.hidden(x),0.2)
            x=self.out(x)
        return x


# disc=Disc(fc=True,n_part=1)
# opt=torch.optim.Adam(disc.parameters())
# sig=nn.Sigmoid()
# losses=[]
# for i in range(100):
#     a=sig(disc(torch.rand(1000,3)))
#     b=sig(disc(torch.rand(1000,3)*100))
#     true=torch.ones(len(a)).reshape(-1,1)
#     fake=torch.zeros(len(b)).reshape(-1,1)
#     best=torch.vstack((a,b))
#     true=torch.vstack((true,fake))
#     loss=nn.BCELoss()(best,true)
#     opt.zero_grad()
#     loss.backward()
#     opt.step()
#     losses.append(loss.detach().numpy())
# plt.plot(losses)


In [8]:
 class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

            def __init__(self, optimizer, warmup, max_iters):
                self.warmup = warmup
                self.max_num_iters = max_iters
                super().__init__(optimizer)

            def get_lr(self):
                lr_factor = self.get_lr_factor(epoch=self.last_epoch)
                return [base_lr * lr_factor for base_lr in self.base_lrs]

            def get_lr_factor(self, epoch):
                lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
                if epoch <= self.warmup:
                    lr_factor *= epoch * 1.0 / self.warmup
                return lr_factor
        
class TransGan(pl.LightningModule):
    def create_resnet(self,in_features, out_features):
        '''This is the network that outputs the parameters of the invertible transformation
        The only arguments can be the in dimension and the out dimenson, the structure
        of the network is defined over the config which is a class attribute
        Context Features: Amount of features used to condition the flow - in our case 
        this is usually the mass
        num_blocks: How many Resnet blocks should be used, one res net block is are 1 input+ 2 layers
        and an additive skip connection from the first to the third'''
        c=self.config["context_features"]
        return nets.ResidualNet(
                in_features,
                out_features,
                hidden_features=self.config["network_nodes_nf"],
                context_features=c,
                num_blocks=self.config["network_layers_nf"],
                activation=self.config["activation"]  if "activation" in self.config.keys() else FF.relu,
                dropout_probability=config["dropout"] if "dropout" in self.config.keys() else 0,
                use_batch_norm=self.config["batchnorm"] if "batchnorm" in self.config.keys() else 0
        )

    def __init__(self,config,hyperopt):
        
        '''This initializes the model and its hyperparameters'''
        super().__init__()
        self.automatic_optimization=False
        self.freq_d=config["freq"]
        self.wgan=config["wgan"]
        self.config=config
        self.counter=0 #This counts how many nan grads we have, we break after 5 in a row
        self.hyperopt=hyperopt
        #Metrics to track during the training
        self.metrics={"val_w1p":[],"val_w1m":[],"val_w1efp":[],"val_cov":[],"val_mmd":[],"val_fpnd":[],"val_logprob":[],"step":[]}
        #Loss function of the Normalizing flows
        self.logprobs=[]
        self.n_part=config["n_part"]
        self.hparams.update(config)
        self.save_hyperparameters()
        #This is the Normalizing flow model to be used later, it uses as many
        #coupling_layers as given in the config 
        self.flows = []
        self.n_dim=self.config["n_dim"]
        self.n_part=config["n_part"]
        K=self.config["coupling_layers"]
        for i in range(K):
            '''This creates the masks for the coupling layers, particle masks are masks
            created such that each feature particle (eta,phi,pt) is masked together or not'''
            if self.config["autoreg"]:
                self.flows += [MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
#                         random_mask=True,
                        features=self.n_dim,
                        hidden_features=128,
                        use_residual_blocks=True, 
                        tails='linear',
                        tail_bound=self.config["tail_bound"],
                        num_bins=self.config["bins"] )]
            else:
                mask=create_random_binary_mask(self.n_dim*self.n_part)            
                self.flows += [PiecewiseRationalQuadraticCouplingTransform(
                            mask=mask,
                            transform_net_create_fn= self.create_resnet, 
                            tails='linear',
                            tail_bound=self.config["tail_bound"],
                            num_bins=self.config["bins"] )]

        self.q0 = nf.distributions.normal.StandardNormal([self.n_dim*self.n_part])
        self.q_test =nf.distributions.normal.StandardNormal([self.n_dim*self.n_part])
        self.flows=CompositeTransform(self.flows)
        self.flow = base.Flow(distribution=self.q0, transform=self.flows)
        self.dis_net = Disc(n_dim=self.n_dim,hidden=config["hidden"],num_layers=config["num_layers"],
                            num_heads=config["heads"],fc=config["fc"],n_part=config["n_part"]).cuda()
        self.alpha=1
        self.sig=nn.Sigmoid()
        self.add_corr=config["corr"]
    def load_datamodule(self,data_module):
        '''needed for lightning training to work, it just sets the dataloader for training and validation'''
        self.data_module=data_module
        

     
    def configure_optimizers(self):
        self.batch_size=self.config["batch_size"]
        self.lr=0.01
        opt_d = torch.optim.Adam(self.dis_net.parameters(), lr=self.lr)
        opt_nf = torch.optim.AdamW(self.flow.parameters(), lr=self.lr)

        return [opt_nf,opt_d]

    def training_step(self, batch, batch_idx):
        """training loop of the model, here all the data is passed forward to a gaussian
            This is the important part what is happening here. This is all the training we do """
        opt_nf,opt_d=self.optimizers()
        nf_loss=0
        d_loss_avg=0
        gradient_penalty=0
        if self.global_step<1200:
            nf_loss -=self.flow.to(self.device).log_prob(batch).mean()#c if self.config["context_features"] else None
            nf_loss/=(self.n_dim*self.n_part) 
            opt_nf.zero_grad()
            self.manual_backward(nf_loss)
            opt_nf.step()
            self.log("logprob", nf_loss, on_step=True, on_epoch=False, prog_bar=True, logger=True) 
        batch=batch.reshape(len(batch),self.n_part,self.n_dim)
        if self.current_epoch>-1:
            with torch.no_grad():
                z=self.flow.sample(len(batch)).reshape(len(batch),self.n_part,self.n_dim)
            fake=z
            pred_real=self.sig(self.dis_net(batch))
            pred_fake=self.sig(self.dis_net(fake.detach()))
            target_real=torch.ones_like(pred_real)
            target_fake=torch.zeros_like(pred_fake)
            d_loss=nn.BCELoss()(pred_fake.view(-1),target_fake.view(-1)).mean()
            d_loss+=nn.BCELoss()(pred_real.view(-1),target_real.view(-1)).mean()
            d_loss/=2
            opt_d.zero_grad()
            self.manual_backward(d_loss)
            opt_d.step()
            d_loss_avg+=d_loss.cpu().detach().numpy()-10*gradient_penalty
            self.log("d_loss",d_loss_avg/self.freq_d,logger=False,prog_bar=True)
            self.logger.experiment.add_scalars("d_losses",{"train_disc":d_loss_avg/self.freq_d},global_step=self.global_step)
            if self.global_step%200==0 :

                    fig,ax=plt.subplots()
                    ax.hist(self.sig(pred_fake).detach().cpu().numpy(),label="fake",bins=np.linspace(0,1,30) if not self.wgan else 30,histtype='step')
                    ax.hist(self.sig(pred_real).detach().cpu().numpy(),label="real",bins=np.linspace(0,1,30) if not self.wgan else 30,histtype='step')
                    ax.legend()
                    plt.ylabel("Counts")
                    plt.xlabel("Score")
                    self.logger.experiment.add_figure("train_class",fig,global_step=self.global_step)

  
        
   
config = {
       "network_layers": 3,  # sets amount hidden layers in transformation networks -scannable
        "network_nodes": 6,  # amount nodes in hidden layers in transformation networks -scannable
        "network_layers_nf": 2,  # sets amount hidden layers in transformation networks -scannable
        "network_nodes_nf": 256,  # amount nodes in hidden layers in transformation networks -scannable
        "batch_size": 5000,  # sets batch size -scannable
        "embedding_features":8,
        "coupling_layers": 5,  # amount of invertible transformations to use -scannable
        "lr": 0.001,  # sets learning rate -scannable
        "batchnorm": False,  # use batchnorm or not -scannable
        "bins": 8,  # amount of bins to use in rational quadratic splines -scannable
        "tail_bound": 6,  # splines:max value that is transformed, over this value theree is id  -scannable
        "limit": 150000,  # how many data points to use, test_set is 10% of this -scannable in a sense use 10 k for faster training
        "n_dim": 3,  # how many dimensions to use or equivalently /3 gives the amount of particles to use NEVER EVER CHANGE THIS
        "dropout": 0.4,  # use droput proportion, for 0 there is no dropout -scannable
        "canonical": False,  # transform data coordinates to px,py,pz -scannable
        "max_steps": 100000,  # how many steps to use at max - lower for quicker training
        "lambda": 10,  # balance between massloss and nll -scannable
        "name": "t",  # name for logging folder
        "disc": False,  # whether to train gan style discriminator that decides whether point is simulated or generated-semi-scannable
        "calc_massloss": False, # whether to calculate mass loss, makes training slower, do not use with autoregressive! 
        "context_features":0, #amount of variables used for conditioning, for 0 no conditioning is used, for 1 o nly the mass is used, for 2 also the number part is used
        "variable":1, #use variable amount of particles otherwise only use 30, options are true or false 
        "parton":"t", #choose the dataset you want to train options: t for top,q for quark,g for gluon
        "oversampling":False,
        "wgan":False,
        "corr":True,
        "num_layers":4,
        "autoreg":False,
        "freq":1,
        "n_part":5,
        "fc":True,
        "hidden":500,
        "heads":2
    }     
print("wtfman")
model=TransGan(config,False)
print("model initialized")

data_module = JetNetDataloader(config) #this loads the data

model.load_datamodule(data_module)#adds datamodule to model
print("data_module loader")
model.config = config #config are our hyperparams, we make this a class property now
logger = TensorBoardLogger("./lightning_logs")
#log every n steps could be important as it decides how often it should log to tensorboard
# Also check val every n epochs, as validation checking takes some time
trainer = pl.Trainer(gpus=1, logger=logger,  log_every_n_steps=10,  # auto_scale_batch_size="binsearch",
                      max_steps=1000000,#,gradient_clip_val=0.01, gradient_clip_algorithm="norm"
                      check_val_every_n_epoch=20 ,num_sanity_val_steps=1,track_grad_norm=2,
                     fast_dev_run=False,max_epochs=-1)
# This calls the fit function which trains the model
trainer.fit(model, train_dataloaders=data_module )  
       
    


/beegfs/desy/user/kaechben/.conda/envs/jetnet/lib/python3.8/site-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


wtfman
model initialized
data_module loader


/beegfs/desy/user/kaechben/.conda/envs/jetnet/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type               | Params
-----------------------------------------------
0 | q0      | StandardNormal     | 0     
1 | q_test  | StandardNormal     | 0     
2 | flows   | CompositeTransform | 1.6 M 
3 | flow    | Flow               | 1.6 M 
4 | dis_net | Disc               | 507 K 
5 | sig     | Sigmoid            | 0     
-----------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.289     Total estimated model params size (MB)
/beegfs/desy/user/kaechben/.conda/envs/jetnet/lib/python3.8/site-packages/pytorch_lightning/trainer/connec

Training: 0it [00:00, ?it/s]

AssertionError: 

In [ ]:
 def validation_step(self, batch, batch_idx):
        '''This calculates some important metrics on the hold out set (checking for overtraining)'''
        batch=batch[:1000,:]
        
        self.data_module.scaler.to("cpu")  
        batch=batch.to("cpu")
        self.dis_net=self.dis_net.train()

        c=None
        
        with torch.no_grad():
#             z=torch.empty((len(batch),self.n_dim))
#             for i in range(self.n_dim//3):
#                 temp=self.flow.sample(len(batch)).detach().cpu()
#                 z[:,3*i:3*(i+1)]=temp
            z=self.flow.sample(len(batch)).detach().cpu()
            z=z.reshape(len(batch),self.n_part,self.n_dim)
            
            # if self.config["oversampling"]:
            #     order=torch.sort(test.reshape(-1,30,3)[:,:,2],dim=1,descending=True)[1]
            #     test=torch.gather(input=test.reshape(-1,30,3),index=order.unsqueeze(-1).repeat(1,1,3),dim=1).reshape(-1,90)
            #test=test.reshape(-1,30,3)[order.repeat(1,1,3)].reshape(-1,90)
            #z+(1-alpha)*
            gen=z
#             if self.add_corr:
#                 gen_corr=z+(1-self.alpha)*self.gen_net.to("cpu")(z)#
#             else:
#                 gen_corr=self.alpha*z.to("cpu")+(1-self.alpha)*self.gen_net.to("cpu")(z)
            gen_corr=gen
            
            
            scores_f=(self.dis_net.to("cpu").train()(z).view(-1))
            scores_t=(self.dis_net.to("cpu").train()(batch.reshape(len(batch),self.n_part,self.n_dim)).view(-1))
            scores_c=(self.dis_net.to("cpu").train()(gen_corr).view(-1))
            
            bins=50
            if not self.wgan:
                bins=np.linspace(0,1,bins)
                scores_f=nn.Sigmoid()(scores_f)
                scores_t=nn.Sigmoid()(scores_t)
                scores_c=nn.Sigmoid()(scores_c)
                real_loss=nn.BCELoss()(scores_t,torch.ones_like(scores_t))
                fake_loss_gan=nn.BCELoss()(scores_c,torch.zeros_like(scores_t))
                fake_loss_nf=nn.BCELoss()(scores_f,torch.zeros_like(scores_t))
                g_loss=nn.BCELoss()(scores_c,torch.ones_like(scores_c))
                num=2
    #                 if self.global_step<100+pretrain:
    #                     fake_loss=0
    #                     num=2
                    #d_loss=(real_loss+(1-self.alpha)*fake_loss_gan+self.alpha*fake_loss_nf)/num
                d_loss=(real_loss+fake_loss_nf)/num
                self.logger.experiment.add_scalars("d_losses",{"val_disc":d_loss},global_step=self.global_step)
                self.logger.experiment.add_scalars("g_losses",{"val_gen":g_loss},global_step=self.global_step)
                
            fig=plt.figure()
            _,bins,_=plt.hist(scores_f.numpy(),bins=bins,label="fake",alpha=0.5)
            plt.hist(scores_t.numpy(),bins=bins,label="true",alpha=0.5)
            plt.hist(scores_c.numpy(),bins=bins,label="corr",alpha=0.5)
            plt.legend()
            self.logger.experiment.add_figure("class_dist",fig,global_step=self.global_step)
            gen=gen[:,:self.n_dim*self.n_part].cpu().detach().reshape(-1,self.n_dim*self.n_part)
            
        # Reverse Standard Scaling (this has nothing to do with flows, it is a standard preprocessing step)

        gen=self.data_module.scaler.inverse_transform(gen)
        gen_corr=self.data_module.scaler.inverse_transform(gen_corr.reshape(-1,self.n_dim*self.n_part))
        true=self.data_module.scaler.inverse_transform(batch[:,:self.n_dim*self.n_part])[:,:self.n_dim*self.n_part]
        m_t=mass(true[:,:self.n_dim*self.n_part].to(self.device),self.config["canonical"]).cpu()
        m_gen=mass(gen[:,:self.n_dim*self.n_part],self.config["canonical"]).cpu()
        m_c=mass(gen_corr[:,:self.n_dim*self.n_part],self.config["canonical"]).cpu()
        self.plot=plotting(model=self,gen=gen[:,:self.n_dim*self.n_part],gen_corr=gen_corr[:,:self.n_dim*self.n_part],true=true[:,:self.n_dim*self.n_part],config=self.config,step=self.global_step,logger=self.logger.experiment)
        self.flow=self.flow.to(self.device)
        self.gen_net=self.gen_net.to("cuda")
        self.dis_net=self.dis_net.to("cuda")
        try:
            self.plot.plot_mass(m=m_gen.cpu().numpy(),m_t=m_t.cpu().numpy(),m_c=m_c.cpu().numpy(),save=True,bins=15,quantile=True,plot_vline=False)
#             self.plot.plot_marginals(save=True)
            self.plot.plot_2d(save=True)
#             self.plot.var_part(true=true[:,:self.n_dim],gen=test[:,:self.n_dim],true_n=n_true,gen_n=n_test,m_true=m_t,m_gen=m_test ,save=True)
        except Exception as e:
            traceback.print_exc()

In [46]:
from helpers import mass
del model
import gc
torch.cuda.empty_cache()
gc.collect()

67686

In [13]:

encoder_layer = nn.TransformerEncoderLayer(d_model=15, nhead=1).cuda()
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6).cuda()
lin=nn.Sequential(*[nn.Linear(99,99) for i in range(6)]).cuda()
import time
error=[]
for i in range(100):
    start=time.time()
    src = torch.rand(5000, 99).cuda()
    out = lin(src).detach()
    error.append(time.time()-start)
    
print(np.average(error))


0.004051065444946289
